# ProxyWhirl Examples

Real-world proxy rotation with live proxies.

**Sections:**
1. [Quick Start](#1-quick-start)
2. [Async Usage](#2-async-usage)
3. [Fetching & Pool Management](#3-fetching--pool-management)
4. [Rotation Strategies](#4-rotation-strategies)
5. [Retry & Resilience](#5-retry--resilience)
6. [CLI](#6-cli)
7. [REST API & MCP](#7-rest-api--mcp)

> All examples use real proxies fetched from public sources. Expect some failures —
> that's real-world behavior, and ProxyWhirl handles it gracefully.

In [ ]:
# !pip -q install -U proxywhirl httpx fastapi

In [ ]:
from __future__ import annotations

import asyncio
import json
import platform
import random
import re
import subprocess
import sys
import threading
from textwrap import shorten

import proxywhirl
from proxywhirl import (
    ALL_SOURCES,
    AsyncProxyWhirl,
    BootstrapConfig,
    Proxy,
    ProxyFetcher,
    ProxyWhirl,
    RetryPolicy,
    configure_logging,
)

# Suppress library-internal logs so notebook output stays clean.
configure_logging(level="WARNING")

print(f"Python {platform.python_version()} \u00b7 ProxyWhirl {proxywhirl.__version__}")

In [ ]:
# \u2500\u2500 Notebook helpers (not part of ProxyWhirl API) \u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500

_ANSI_RE = re.compile(r"\x1b\[[0-9;]*m")


def run_async(coro):
    """Run async code in notebooks with an already-running event loop."""
    try:
        asyncio.get_running_loop()
    except RuntimeError:
        return asyncio.run(coro)

    result: dict[str, object] = {}
    error: dict[str, BaseException] = {}

    def _runner() -> None:
        try:
            result["value"] = asyncio.run(coro)
        except BaseException as exc:
            error["value"] = exc

    thread = threading.Thread(target=_runner, daemon=True)
    thread.start()
    thread.join()

    if "value" in error:
        raise error["value"]
    return result.get("value")


def jprint(value) -> None:
    """Pretty-print a dict/list as indented JSON."""
    print(json.dumps(value, indent=2, default=str))


def run_cli(args: list[str], max_lines: int = 14) -> int:
    """Run a proxywhirl CLI command and display output."""
    cmd = [sys.executable, "-m", "proxywhirl.cli"] + args
    print(f"$ proxywhirl {' '.join(args)}")
    completed = subprocess.run(cmd, capture_output=True, text=True)

    if completed.stdout:
        clean = _ANSI_RE.sub("", completed.stdout).strip()
        lines = clean.splitlines()
        print("\n".join(lines[:max_lines]))
        if len(lines) > max_lines:
            print("...")

    if completed.stderr:
        err = _ANSI_RE.sub("", completed.stderr).strip()
        if err:
            print("[stderr]", shorten(err, width=220, placeholder=" ..."))

    print(f"(exit={completed.returncode})\n")
    return completed.returncode

## 1. Quick Start

The simplest usage — auto-fetches proxies from public sources on the first request.
Default bootstrap randomly samples from all available sources.

In [ ]:
rotator = ProxyWhirl()  # Zero config \u2014 auto-bootstraps from ALL_SOURCES on first request

try:
    response = rotator.get("https://httpbin.org/ip")
    print(f"Status: {response.status_code}")
    jprint(response.json())
except Exception as e:
    print(f"Request failed (common with free proxies): {type(e).__name__}: {e}")

print(f"\nPool size: {rotator.pool.size}")
jprint(rotator.get_pool_stats())

In [ ]:
# Disable bootstrap \u2014 bring your own proxies
manual = ProxyWhirl(bootstrap=False)
manual.add_proxy("http://my-proxy.example.com:8080")
manual.add_proxy("http://my-proxy2.example.com:9090")
print(f"Manual pool: {manual.pool.size}")

# Fine-tune bootstrap \u2014 more sources, cap proxy count
config = BootstrapConfig(
    sample_size=20,
    validate_proxies=True,
    max_proxies=50,
    show_progress=True,
)
print(
    f"Custom: sample {config.sample_size} sources, "
    f"validate={config.validate_proxies}, max {config.max_proxies} proxies"
)

## 2. Async Usage

Use `AsyncProxyWhirl` in async frameworks (FastAPI, aiohttp). Same API shape as sync.

In [ ]:
async def async_demo():
    async with AsyncProxyWhirl() as rotator:
        try:
            response = await rotator.get("https://httpbin.org/ip")
            return response.json(), rotator.get_pool_stats()
        except Exception as e:
            return {"error": str(e)}, rotator.get_pool_stats()


payload, stats = run_async(async_demo())
jprint(payload)
print(f"\nPool: {stats['total_proxies']} proxies, {stats['healthy_proxies']} healthy")

## 3. Fetching & Pool Management

Fetch proxies from specific sources, inspect the pool, and clean up dead proxies.

In [ ]:
async def fetch_demo():
    # Sample random sources from ALL_SOURCES for variety
    sources = random.sample(ALL_SOURCES, min(5, len(ALL_SOURCES)))
    async with ProxyFetcher(sources=sources) as fetcher:
        return await fetcher.fetch_all(validate=False, deduplicate=True)


raw_proxies = run_async(fetch_demo())
print(f"Fetched {len(raw_proxies)} unique proxies")
for p in raw_proxies[:5]:
    print(f"  {p['url']}")
if len(raw_proxies) > 5:
    print(f"  ... and {len(raw_proxies) - 5} more")

In [ ]:
fetched_rotator = ProxyWhirl(bootstrap=False)
for p in raw_proxies[:20]:
    fetched_rotator.add_proxy(p["url"])

print(f"Pool: {fetched_rotator.pool.size} proxies")
jprint(fetched_rotator.get_statistics())

In [ ]:
print(f"Before cleanup: {rotator.pool.size} proxies")
jprint(rotator.get_pool_stats())

removed = rotator.clear_unhealthy_proxies()
print(f"\nRemoved {removed} unhealthy/dead proxies")
print(f"After cleanup: {rotator.pool.size} proxies")

## 4. Rotation Strategies

ProxyWhirl supports 7 rotation strategies via string aliases. Swap at runtime without restarting.

In [ ]:
from proxywhirl.strategies import (
    GeoTargetedStrategy,
    LeastUsedStrategy,
    PerformanceBasedStrategy,
    RandomStrategy,
    RoundRobinStrategy,
    SessionPersistenceStrategy,
    WeightedStrategy,
)

STRATEGY_MAP = {
    "round-robin": RoundRobinStrategy,
    "random": RandomStrategy,
    "weighted": WeightedStrategy,
    "least-used": LeastUsedStrategy,
    "performance-based": PerformanceBasedStrategy,
    "session": SessionPersistenceStrategy,
    "geo-targeted": GeoTargetedStrategy,
}

print("Available strategies:")
for name, cls in STRATEGY_MAP.items():
    print(f"  {name:18} -> {cls.__name__}")

In [ ]:
print(f"Current: {rotator.strategy.__class__.__name__}")
rotator.set_strategy("performance-based")
print(f"Swapped: {rotator.strategy.__class__.__name__}")

for i in range(3):
    try:
        resp = rotator.get("https://httpbin.org/ip")
        origin = resp.json().get("origin", "?")
        print(f"  Request {i + 1}: {resp.status_code} via {origin}")
    except Exception as e:
        print(f"  Request {i + 1}: {type(e).__name__}")

rotator.set_strategy("round-robin")
print(f"Restored: {rotator.strategy.__class__.__name__}")

## 5. Retry & Resilience

Configure automatic retries with exponential backoff.

In [ ]:
policy = RetryPolicy(
    max_attempts=3,
    base_delay=0.5,
    jitter=True,
    retry_status_codes=[502, 503, 504],
)

print(f"Max attempts: {policy.max_attempts}")
print(f"Backoff: {policy.backoff_strategy.value}, base_delay={policy.base_delay}s")
print(f"Retry on: {policy.retry_status_codes}")

# Show deterministic delay schedule (jitter adds randomization on top)
base_delays = [round(policy.base_delay * (2.0**i), 2) for i in range(policy.max_attempts)]
print(f"Base delays: {base_delays}s (randomized by jitter in practice)")

resilient = ProxyWhirl(retry_policy=policy, strategy="random")

try:
    response = resilient.get("https://httpbin.org/ip")
    print(f"\nSuccess: {response.status_code}")
    jprint(response.json())
except Exception as e:
    print(f"\nAll retries exhausted: {type(e).__name__}: {e}")

summary = resilient.get_retry_metrics().get_summary()
print(f"\nRetry metrics: {summary['total_retries']} attempts tracked")

## 6. CLI

Run proxywhirl from the command line.

In [ ]:
run_cli(["--help"], max_lines=20)
run_cli(["--no-lock", "config", "show"], max_lines=16)
run_cli(["--no-lock", "stats"], max_lines=14)

## 7. REST API & MCP

Expose ProxyWhirl as a REST API or use it with AI assistants via MCP.

In [ ]:
from fastapi.testclient import TestClient

from proxywhirl.api import app, get_rotator

api_rotator = ProxyWhirl(
    bootstrap=False,
    proxies=[Proxy(url=p["url"]) for p in raw_proxies[:10]],
)
app.dependency_overrides[get_rotator] = lambda: api_rotator

with TestClient(app) as client:
    status = client.get("/api/v1/status").json()
    proxies_resp = client.get("/api/v1/proxies").json()

app.dependency_overrides.clear()

print("Status:")
jprint(status.get("data", {}))
print(f"\nProxies: {proxies_resp.get('data', {}).get('total', 0)} loaded")
first = (proxies_resp.get("data", {}).get("items") or [None])[0]
if first:
    jprint(first)

In [ ]:
from loguru import logger

logger.disable("proxywhirl.mcp.server")
from proxywhirl.mcp.server import (  # noqa: E402, I001
    cleanup_rotator,
    get_proxy_health,
    list_proxies,
    rotate_proxy,
    set_rotator,
)

logger.enable("proxywhirl.mcp.server")


async def mcp_demo():
    mcp_rotator = AsyncProxyWhirl(
        bootstrap=False,
        proxies=[Proxy(url=p["url"]) for p in raw_proxies[:10]],
    )
    await set_rotator(mcp_rotator)

    listing = await list_proxies()
    rotated = await rotate_proxy()
    health = await get_proxy_health()

    await cleanup_rotator()
    return listing, rotated, health


mcp_listing, mcp_rotated, mcp_health = run_async(mcp_demo())
print(f"list_proxies: {mcp_listing.get('total', 0)} proxies")
print("\nrotate_proxy:")
jprint(mcp_rotated)
print("\nhealth:")
jprint(json.loads(mcp_health))

## Next Steps

- **Docs**: See the [documentation](docs/) for API reference and guides
- **CLI**: Run `proxywhirl --help` for all commands
- **TUI**: Launch `proxywhirl tui` for a live monitoring dashboard
- **MCP**: Run `proxywhirl-mcp` to expose rotation to AI assistants